In [1]:
from pytrends.request import TrendReq
from addon import *
import pandas as pd
import time
import pycountry

In [8]:

pytrends = TrendReq(hl='en-US', tz=360)

## %
kw_list = ["covid depression", "covid health insurance", "test sites", "bars near me"]
df = pd.DataFrame()

In [9]:
states = [ 'AL', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL', 'GA', 'IL', 'IN', 'KS', 'KY', 'LA', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [10]:
i = 0
while i < len(states):
    try:
        pytrends.build_payload(kw_list, cat=0, timeframe='2020-03-22 2020-10-09', geo="US-" + states[i], gprop='')
        
        state_df = pytrends.interest_over_time()
        state_df["State"] = states[i]
        
        df = pd.concat([df, state_df])
        time.sleep(.1)

        print(states[i])
        i += 1

        time.sleep(3)
    except Exception as e:
        i -= 1
        time.sleep(5)
        print(e)


AL
AZ
AR
CA
CO
CT
DE
DC
FL
GA
IL
IN
KS
KY
LA
MD
MA
MI
MN
MS
MO
MT
NE
NV
NH
NJ
NM
NY
NC
ND
OK
OR
PA
RI
SC
SD
TN
TX
VT
VA
WA
WV
WI
WY


In [11]:
df.to_csv("../../data/trends/final_data/USA_statewise_4.csv")

In [38]:
df1 = pd.read_pickle("../../df1.pkl")
df2 = pd.read_pickle("../../df2.pkl")
df3 = pd.read_pickle("../../df3.pkl")
df4 = pd.read_pickle("../../df4.pkl")

df_combined = pd.concat([df1, df2, df3, df4])
df_combined = df_combined.sort_values(by='date')
df_combined["date"] = df_combined["date"] + " 00:00:01"
df_combined["date"] = pd.to_datetime(df_combined["date"])
print(df_combined)
df_combined.to_csv("../../data/hopkins_data/df_new_2.csv")

                    date     FIPS   Admin2   Province_State Country_Region  \
75   2020-03-22 00:00:01  34003.0   Bergen       New Jersey             US   
2875 2020-03-22 00:00:01  48353.0    Nolan            Texas             US   
1103 2020-03-22 00:00:01  16081.0    Teton            Idaho             US   
2804 2020-03-22 00:00:01  48169.0    Garza            Texas             US   
1043 2020-03-22 00:00:01  40049.0   Garvin         Oklahoma             US   
...                  ...      ...      ...              ...            ...   
233  2020-10-09 00:00:01      NaN      NaN              NaN        Iceland   
512  2020-10-09 00:00:01      NaN      NaN    Kurgan Oblast         Russia   
1081 2020-10-09 00:00:01  13109.0    Evans          Georgia             US   
3044 2020-10-09 00:00:01  72103.0  Naguabo      Puerto Rico             US   
477  2020-10-09 00:00:01      NaN      NaN  Adygea Republic         Russia   

              Last_Update        Lat       Long_  Confirmed  De

In [47]:
pop_df = pd.read_csv("../../data/co-est2019-alldata.csv", encoding='latin-1')
df_combined = df_combined.dropna(subset=["FIPS", "Lat"])
df_combined["FIPS"] = df_combined["FIPS"].astype('int64')
print(pop_df.dtypes)
print(df_combined.dtypes)

STATE               int64
COUNTY              int64
STNAME             object
CTYNAME            object
POPESTIMATE2019     int64
FIPS                int64
dtype: object
date                   datetime64[ns]
FIPS                            int64
Admin2                         object
Province_State                 object
Country_Region                 object
Last_Update                    object
Lat                           float64
Long_                         float64
Confirmed                       int64
Deaths                          int64
Recovered                       int64
Active                        float64
Combined_Key                   object
Incidence_Rate                float64
Case-Fatality_Ratio           float64
dtype: object


In [52]:
df_final = pd.merge(df_combined, pop_df, how="outer", on=['FIPS'])
df_final["Confirmed"] = df_final["Confirmed"] / df_final["POPESTIMATE2019"] * 100000
df_final["Deaths"] = df_final["Deaths"] / df_final["POPESTIMATE2019"] * 100000
df_final.to_csv("../../data/hopkins_data/hopkins_population.csv")